## Introducción

En el siguiente proyecto trabajaré con la base de datos de una compañía móvil en la cual analizaré el comportamiento de sus clientes con el objetivo de recomendar un nuevo plan. Para realizar estas recomendaciones, desarrollaré un modelo de clasificación que escoja correctamente cual de los planes se ajusta mejor a cada usuario de acuerdo con sus características. 

## 1. Incialización

In [1]:
#importar librerías
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [2]:
#librerías para elegir un modelo
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## 1.1 Cargar datos

In [3]:
#cargar dataset
users_behav = pd.read_csv("datasets/users_behavior.csv")

In [4]:
#visualizar informacion general del dataset
users_behav.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
#visualizar datos
users_behav.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## 2. Segmentar los datos para el modelo

In [6]:
#definir variables para las características y el objeto del modelo (conjunto de prueba)
features = users_behav.drop(['is_ultra'], axis=1)
target = users_behav['is_ultra']

#segmentar un conjunto de entranamiento y uno de validación 
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.20, random_state=12345)

## 3. Comparar la calidad de los modelos

In [7]:
#árbol de decisión de regresión
best_model = None
best_result = 1
best_depth = 0

for depth in range (1,6):
    model = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    prediction_valid = model.predict(features_valid)

    result = mean_squared_error(target_valid, prediction_valid) ** 0.5

    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth


print(f"RECM del mejor modelo en el conjunto de validación (max_depth = {best_depth}): {best_result}")
 

RECM del mejor modelo en el conjunto de validación (max_depth = 5): 0.4015534785850737


In [8]:
#RandomForest regresor
best_model = None
best_result = 1
best_depth = 0
best_est = 0

for est in range(5, 30, 50,):
    for depth in range(1,10):
        model = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        
        result = mean_squared_error(target_valid, prediction_valid) ** 0.5

        if result < best_result:
            best_model = model
            best_result = result
            best_depth = depth
            best_est = est


print(f"RECM del mejor modelo en el conjunto de validación= {best_result}, n_estimators= {best_est}, best_depth= {best_depth}")


RECM del mejor modelo en el conjunto de validación= 0.4015534785850737, n_estimators= 5, best_depth= 1
